In [5]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely.wkt import loads

In [7]:
dfCrime = pd.read_csv('/Users/austincoffelt/Documents/Rent-Walk-Local/Calls_for_Service_2019_20250428.csv')
print(dfCrime.shape)
dfCrime.head()

(487362, 21)


,NOPD_Item,Type,TypeText,Priority,MapX,MapY,TimeCreate,TimeDispatch,TimeArrival,TimeClosed,...,DispositionText,InitialType,InitialTypeText,InitialPriority,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
0,E3696819,21P,LOST PROPERTY,0G,3682079,532392,05/26/2019 05:08:29 AM,05/26/2019 05:08:29 AM,05/26/2019 05:08:29 AM,05/26/2019 05:24:02 AM,...,REPORT TO FOLLOW,21,COMPLAINT OTHER,1J,Y,8D06,006XX Bourbon St,70130.0,8,"(29.95787779, -90.06620983)"
1,E3711919,34D,AGGRAVATED BATTERY DOMESTIC,2C,3674751,525920,05/26/2019 09:10:51 AM,05/26/2019 09:12:32 AM,05/26/2019 09:21:11 AM,05/27/2019 05:49:50 PM,...,REPORT TO FOLLOW,34S,AGGRAVATED BATTERY BY SHOOTING,2C,N,6I02,025XX 2nd St,70113.0,6,"(29.94030144, -90.08957166)"
2,E3721919,21P,LOST PROPERTY,0G,3681704,531945,05/26/2019 10:47:30 AM,05/26/2019 03:46:05 PM,05/26/2019 03:46:10 PM,05/26/2019 04:42:53 PM,...,REPORT TO FOLLOW,67,THEFT,1G,N,8D05,004XX Blk Bourbon St,70112.0,8,"(29.95666057, -90.06741082)"
3,E3724019,103D,DOMESTIC DISTURBANCE,2C,3699162,557853,05/26/2019 11:12:23 AM,05/26/2019 11:18:05 AM,05/26/2019 11:39:08 AM,05/26/2019 01:06:41 PM,...,REPORT TO FOLLOW,103D,DOMESTIC DISTURBANCE,2C,N,7D06,070XX Martin Dr,70126.0,7,"(30.02735574, -90.01133827)"
4,E3735719,62C,SIMPLE BURGLARY VEHICLE,1G,3727938,560667,05/26/2019 01:36:51 PM,05/26/2019 05:37:42 PM,05/26/2019 05:37:47 PM,05/26/2019 06:36:35 PM,...,REPORT TO FOLLOW,21P,LOST PROPERTY,0G,N,7L02,138XX Chef Menteur Hwy,70129.0,7,"(30.03414756, -89.92030106)"


In [8]:
sorted(list(dfCrime['TypeText'].unique()))

['ABANDONED BOAT',
 'ABANDONED VEHICLE',
 'AGGRAVATED ARSON',
 'AGGRAVATED ASSAULT',
 'AGGRAVATED ASSAULT DOMESTIC',
 'AGGRAVATED BATTERY',
 'AGGRAVATED BATTERY BY CUTTING',
 'AGGRAVATED BATTERY BY SHOOTING',
 'AGGRAVATED BATTERY DOMESTIC',
 'AGGRAVATED BURGLARY',
 'AGGRAVATED BURGLARY DOMESTIC',
 'AGGRAVATED CRIMINAL DAMAGE',
 'AGGRAVATED KIDNAPPING',
 'AGGRAVATED RAPE',
 'AGGRAVATED RAPE MALE VICTIM',
 'AGGRAVATED RAPE UNFOUNDED BY SPECIAL VICTIMS OR CHILD ABUSE',
 'AIRPLANE CRASH',
 'AREA CHECK',
 'ARMED ROBBERY',
 'ARMED ROBBERY WITH GUN',
 'ARMED ROBBERY WITH KNIFE',
 'ASSET SEIZURE',
 'ATTACHMENT',
 'AUTO ACCIDENT',
 'AUTO ACCIDENT CITY VEHICLE',
 'AUTO ACCIDENT FATALITY',
 'AUTO ACCIDENT POLICE VEHICLE',
 'AUTO ACCIDENT WITH INJURY',
 'AUTO THEFT',
 'BICYCLE THEFT',
 'BLIGHTED PROPERTY',
 'BOMB SCARE',
 'BURGLAR ALARM, LOCAL',
 'BURGLAR ALARM, SILENT',
 'BURGLAR ALARM, SILENT, FAR NO RESPONSE',
 'BUSINESS BURGLARY',
 'BUSINESS CHECK',
 'CAD TEST',
 'CARJACKING',
 'CARJACKING- NO

In [9]:
violentCrimes = ['AGGRAVATED ASSAULT', 'AGGRAVATED ASSAULT DOMESTIC', 'AGGRAVATED RAPE', 'AGGRAVATED RAPE MALE VICTIM', 
                 'AGGRAVATED RAPE UNFOUNDED BY SPECIAL VICTIMS OR CHILD ABUSE', 'ARMED ROBBERY', 'ARMED ROBBERY WITH GUN', 
                 'ARMED ROBBERY WITH KNIFE', 'HOMICIDE', 'HOMICIDE BY CUTTING', 'HOMICIDE BY SHOOTING', 'HOMICIDE DOMESTIC',
                'SIMPLE RAPE', 'SIMPLE RAPE MALE VICTIM', 'SIMPLE RAPE UNFOUNDED BY SPECIAL VICTIMS OR CHILD ABUSE']
dfCrime = dfCrime[dfCrime['TypeText'].isin(violentCrimes)]
print(dfCrime.shape)
dfCrime.head()

(2694, 21)


,NOPD_Item,Type,TypeText,Priority,MapX,MapY,TimeCreate,TimeDispatch,TimeArrival,TimeClosed,...,DispositionText,InitialType,InitialTypeText,InitialPriority,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
15,E3779419,30,HOMICIDE,2C,3700961,552103,05/26/2019 09:02:13 PM,05/26/2019 09:04:23 PM,05/26/2019 09:04:30 PM,05/27/2019 12:00:46 AM,...,REPORT TO FOLLOW,35,SIMPLE BATTERY,1F,N,7C02,Chantilly Dr & Chef Menteur Hwy,70126.0,7,"(30.01148679, -90.0058646)"
27,E3785919,30S,HOMICIDE BY SHOOTING,2C,3683148,540254,05/26/2019 09:58:47 PM,05/26/2019 10:00:07 PM,05/26/2019 10:05:44 PM,05/27/2019 01:47:39 AM,...,REPORT TO FOLLOW,34S,AGGRAVATED BATTERY BY SHOOTING,2C,N,5K02,022XX Saint Anthony St,70119.0,5,"(29.97946128, -90.06256029)"
91,E3801619,30S,HOMICIDE BY SHOOTING,2C,3681380,536536,05/26/2019 11:55:43 PM,05/26/2019 11:56:23 PM,05/26/2019 11:56:29 PM,05/27/2019 03:34:54 AM,...,REPORT TO FOLLOW,34S,AGGRAVATED BATTERY BY SHOOTING,2C,N,1A01,014XX N Claiborne Ave,70116.0,1,"(29.96929168, -90.06827204)"
129,E3805419,42,AGGRAVATED RAPE,1A,37369000,3513814,05/27/2019 12:40:59 AM,05/27/2019 01:29:48 AM,05/27/2019 01:42:48 AM,05/27/2019 03:49:51 AM,...,REPORT TO FOLLOW,42,AGGRAVATED RAPE,2C,N,7O04,070XX Crowder Blvd,70126.0,7,"(0.0, 0.0)"
447,D3751119,37,AGGRAVATED ASSAULT,1A,3686233,534510,04/27/2019 07:20:36 PM,04/27/2019 11:52:48 PM,04/28/2019 12:04:05 AM,04/28/2019 12:07:38 AM,...,GONE ON ARRIVAL,37,AGGRAVATED ASSAULT,2E,N,5B01,005XX Spain St,70117.0,5,"(29.96357366, -90.05301905)"


In [10]:
dfCrime = dfCrime[['TypeText', 'Location']]
dfCrime = dfCrime[dfCrime['Location'] != '(0.0, 0.0)']
dfCrime = dfCrime[dfCrime['Location'].isna() == False]
dfCrime.head()

,TypeText,Location
15,HOMICIDE,"(30.01148679, -90.0058646)"
27,HOMICIDE BY SHOOTING,"(29.97946128, -90.06256029)"
91,HOMICIDE BY SHOOTING,"(29.96929168, -90.06827204)"
447,AGGRAVATED ASSAULT,"(29.96357366, -90.05301905)"
455,HOMICIDE BY SHOOTING,"(29.97133993, -90.11667108)"


In [11]:
dfCrime.dtypes

TypeText    object
Location    object
dtype: object

In [12]:
# Extract lat and lon using regex (handles parentheses and commas)
dfCrime[['lat', 'lon']] = dfCrime['Location'].str.extract(r'\(([^,]+),\s*([^)]+)\)').astype(float)

# Create Point geometry
dfCrime['geometry'] = dfCrime.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

dfCrime = dfCrime[['Location', 'geometry']]

In [19]:
# Read shapefile (automatically loads .shp, .shx, .dbf)
gdf = gpd.read_file("/Users/austincoffelt/Documents/Rent-Walk-Local/cb_2018_22_bg_500k/cb_2018_22_bg_500k.shp")

# View the first 5 rows
print(gdf.shape)
gdf.head()

(3463, 11)


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,22,051,021801,1,1500000US220510218011,220510218011,1,BG,759364,0,"POLYGON ((-90.187 30.012, -90.182 30.012, -90...."
1,22,051,027703,2,1500000US220510277032,220510277032,2,BG,371159,0,"POLYGON ((-90.118 29.848, -90.117 29.848, -90...."
2,22,009,030100,1,1500000US220090301001,220090301001,1,BG,562491448,34183405,"POLYGON ((-92.053 31.162, -92.048 31.168, -92...."
3,22,001,961000,2,1500000US220019610002,220019610002,2,BG,590930,0,"POLYGON ((-92.372 30.209, -92.371 30.209, -92...."
4,22,079,012100,1,1500000US220790121001,220790121001,1,BG,1179564,0,"POLYGON ((-92.463 31.291, -92.463 31.292, -92...."


In [21]:
# make sure points match the crs of gdf
gdfPoints = gpd.GeoDataFrame(
    dfCrime, 
    geometry='geometry',
    crs="EPSG:4269"  # WGS84 coordinate reference system
)
gdfPoints.head()

,Location,geometry
15,"(30.01148679, -90.0058646)",POINT (-90.006 30.011)
27,"(29.97946128, -90.06256029)",POINT (-90.063 29.979)
91,"(29.96929168, -90.06827204)",POINT (-90.068 29.969)
447,"(29.96357366, -90.05301905)",POINT (-90.053 29.964)
455,"(29.97133993, -90.11667108)",POINT (-90.117 29.971)


In [23]:
# Basic spatial join (points within polygons)
result = gpd.sjoin(
    gdfPoints, 
    gdf,
    how="left",       # 'left', 'right', or 'inner'
    predicate="within" # Spatial relationship
)
print(result.shape)
result.head()

(2000, 13)


,Location,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
15,"(30.01148679, -90.0058646)",POINT (-90.006 30.011),2041.0,22,071,001751,1,1500000US220710017511,220710017511,1,BG,42147730.0,4778214.0
27,"(29.97946128, -90.06256029)",POINT (-90.063 29.979),83.0,22,071,003000,2,1500000US220710030002,220710030002,2,BG,190414.0,0.0
91,"(29.96929168, -90.06827204)",POINT (-90.068 29.969),1008.0,22,071,003400,2,1500000US220710034002,220710034002,2,BG,202640.0,0.0
447,"(29.96357366, -90.05301905)",POINT (-90.053 29.964),1466.0,22,071,001800,2,1500000US220710018002,220710018002,2,BG,244189.0,64665.0
455,"(29.97133993, -90.11667108)",POINT (-90.117 29.971),2719.0,22,071,007605,2,1500000US220710076052,220710076052,2,BG,283575.0,0.0


In [25]:
crime = result['AFFGEOID'].value_counts()
crime

AFFGEOID
1500000US220710134003    65
1500000US220710017511    45
1500000US220710134002    42
1500000US220710017204    35
1500000US220710135003    28
                         ..
1500000US220710121021     1
1500000US220710017021     1
1500000US220710065002     1
1500000US220710090001     1
1500000US220710115002     1
Name: count, Length: 392, dtype: int64

In [27]:
# export as a csv
crime.to_csv('/Users/austincoffelt/Documents/RW Data/NO_Violent_Crime_BG.csv')